<a href="https://colab.research.google.com/github/Mburu-Elvis/animal_classification/blob/main/CNN_animal_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to Google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Libraries

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Define the data generators for training and validation sets
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
URL = '/content/drive/MyDrive/Colab Notebooks/Data/animal_dataset'

# Generate the training dataset and validation dataset using the generators
train_dataset = train_datagen.flow_from_directory(
    URL + "/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical', # set the class_mode to 'categorical'
    subset='training'
)

val_dataset = val_datagen.flow_from_directory(
    URL + "/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical', # set the class_mode to 'categorical'
    subset='validation'
)

# Load the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add a new classification layer on top of the pre-trained model
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
output = Dense(80, activation='softmax')(x)

# Create a new model with the pre-trained model as the base and the new classification layer on top
model = Model(inputs=base_model.input, outputs=output)

# Compile the model with categorical crossentropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
try:
    history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)
except tf.errors.InvalidArgumentError as e:
    if 'UnidentifiedImageError' in str(e):
        filename = str(e).split("cannot identify image file '")[1].split("'")[0]
        print(f"Error: Could not identify image file {filename}")
    else:
        raise e


Found 18099 images belonging to 80 classes.
Found 4486 images belonging to 80 classes.
Epoch 1/10
550/566 [============================>.] - ETA: 5:06 - loss: 3.0811 - accuracy: 0.2754

UnknownError: ignored